In [ ]:
class GenerationStrategy:
    def __init__(self, p):
        self.p = p
        
    def random_sequence(length):
        all_ = [s for s in range(256) if not (s & (Select | Start))]
        sequence = np.array(all_)[stats.randint(0, len(all_)).rvs(length)]
        
        return sequence

class MixtureStrategy(GenerationStrategy):
    def generate(pool):
        num_instances, num_actions = pool.shape
        
        keys = stats.zipf(stats.uniform(1.01, 10).rvs(), -1).rvs(num_actions) % num_instances
        keys = num_actions - 1 - keys
        # keys = stats.randint(0, num_instances).rvs(num_actions)
        mixture = np.array([pool[key, i] for i, key in enumerate(keys)])
        
        return mixture

class UniformMutationStrategy(GenerationStrategy):
    def generate(pool):
        num_actions = pool.shape[1]
        mutation = np.choose(stats.bernoulli(self.p).rvs(length),
                             [self.random_sequence(num_actions),
                              pool[pool_index]])
        return mutation
    
class SequentialMutationStrategy(GenerationStrategy):
    def generate(pool):
        num_actions = pool.shape[1]
        
        pool_index = stats.randint(0, len(pool)).rvs()
        
        choices = np.cumsum(stats.bernoulli(self.p).rvs(num_actions)) % 2
        mutation = np.choose(choices,
                             [self.random_sequence(num_actions),
                              pool[pool_index]])
        
        return mutation

class SequentialMixtureStrategy(GenerationStrategy):
    def generate(pool):
        num_actions = pool.shape[1]
        
        pool_indices = stats.randint(0, len(pool)).rvs(2)
        
        choices = np.cumsum(stats.bernoulli(self.p).rvs(num_actions)) % 2
        mutation = np.choose(choices,
                             pool[pool_indices])

In [ ]:
import NESSolve
import numpy as np
import scipy.stats as stats
import matplotlib.pyplot as plt
%config InlineBackend.figure_format='retina'

A      = 0b1000_0000
B      = 0b0100_0000
Select = 0b0010_0000
Start  = 0b0001_0000
Up     = 0b0000_1000
Down   = 0b0000_0100
Left   = 0b0000_0010
Right  = 0b0000_0001
None_  = 0b0000_0000

def random_sequence(length, pool, p):
    all_ = [s for s in range(256) if not (s & (Select | Start))]
    sequence = np.array(all_)[stats.randint(0, len(all_)).rvs(length)]

    if pool == []:
        return sequence
    else:
        pool_index = stats.randint(0, len(pool)).rvs()
        actions_ = np.array(pool)

        keys = stats.randint(0, actions_.shape[0]).rvs(actions_.shape[1])
        mixture = [actions_[key, i] for i, key in enumerate(keys)]
        
        if stats.bernoulli(p).rvs(1)[0]:
            return mixture
        else:
            return sequence
        
        return np.choose(stats.bernoulli(p).rvs(length),
                         [sequence,
                          pool[pool_index]])
    
def frame_skip(sequence, n):
    return sum([[action]*n for action in sequence], [])
    
random_sequence(length=505, pool=[], p=0.5);

In [ ]:
import os
def beep():
    os.system("echo -n '\a'")

In [ ]:
def iteration(length, pool, p):
    k = 2
    sec = 60
    num_instances = k*1024
    num_blocks = k*32

    start_sequences = np.array([[None_]*35 +
                                [Start]*45 +
                                [None_]*40]*num_instances, dtype=np.uint8)
    
    actions = np.array([random_sequence(length, [actions_ for actions_, score in pool], p=p)
                        for _ in range(num_instances)], dtype=np.uint8)
    
    full_sequences = np.concatenate([start_sequences,
                                     actions], axis=1)
    
    if pool == []:
        print(full_sequences.shape[1] / 60)
        print(full_sequences, flush=True)
        
    data_lines, red, green, blue = NESSolve.run("roms/Super Mario Bros..nes", full_sequences, num_blocks)
    
    block_scores = data_lines.reshape(-1, k*1024).transpose()
    actions_scores = [(action_sequence, len(np.unique(score))) for action_sequence, score in zip(actions, block_scores)]
    
    return actions_scores

# pool = []
for i in range(0):
    print(i)
    actions_scores = iteration(60*2, pool, .8)
    pool = sorted(pool + actions_scores, key=lambda x: x[1])
    pool = pool[-2*1024:]
    scores = [score for _, score in pool]
    print(min(scores), scores[-10:])
    
beep()

In [ ]:
import pickle
pool = pickle.load(open('loop.pickle', 'rb'))

In [ ]:
import pickle
pickle.dump(pool, open('loop.pickle', 'wb'))

In [ ]:
for i in range(100):
    print(i)
    actions_scores = iteration(60*10, pool, .7)
    pool = sorted(pool + actions_scores, key=lambda x: x[1])
    pool = pool[-2*1024:]
    scores = [score for _, score in pool]
    print(min(scores), scores[-10:]) 
    
    import pickle
    pickle.dump(pool, open('loop.pickle', 'wb'))

In [ ]:
# %%time

k = 1
sec = 60
num_instances = k*1024
num_blocks = k*32

base = ([None_]*35 +
        [Start]*45 +
        [None_]*40)

actions = [base + list(random_sequence(60*30, [], 0)) for _ in range(num_instances)]
actions = [base + list(sequence) for sequence, score in pool[-1024:]]
actions = np.array(actions, dtype=np.uint8)
print(actions.shape[1] / 60)
print(actions, flush=True)

# data_lines, red, green, blue = NESSolve.run("roms/Donkey Kong.nes", actions, num_blocks)
# red, green, blue = NESSolve.run("roms/digdug.nes", actions, num_blocks)
data_lines, red, green, blue = NESSolve.run("roms/Super Mario Bros..nes", actions, num_blocks)
# red, green, blue = NESSolve.run("roms/Spelunker (USA).nes", actions, num_blocks)
# array = NESSolve.run("roms/01-basics.nes", actions, num_blocks)

reds = red.reshape(240*256, num_instances).transpose().reshape(num_instances, 240, 256)
greens = green.reshape(240*256, num_instances).transpose().reshape(num_instances, 240, 256)
blues = blue.reshape(240*256, num_instances).transpose().reshape(num_instances, 240, 256)

beep();

a = np.array([reds, greens, blues])
print(a.shape)
image = a.transpose(1, 2, 3, 0)[22]
plt.imshow(image);

In [ ]:
%%time

data = data_lines.reshape(-1, k*1024).transpose()
print(data.shape)

arr = [(index, len(np.unique(instance))) for index, instance in enumerate(data)]

sorted_ = sorted(arr, key=lambda x: x[1])

# sorted_ = sorted_[:50] + sorted_[-400:]
sorted_keys = [index for index, score in sorted_]
sorted_scores = [score for index, score in sorted_]
array_sorted = np.array([reds[sorted_keys], greens[sorted_keys], blues[sorted_keys]])
sorted_

In [ ]:
import pickle
pickle.dump([array_sorted, sorted_scores], open('active.pickle', 'wb'))

In [ ]:
beep()